In [1]:
import os
import sys
import json
import albumentations as A

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from building_footprint_segmentation.segmentation import init_segmentation, read_trainer_config
from building_footprint_segmentation.helpers.callbacks import CallbackList, load_callback
from building_footprint_segmentation.trainer import Trainer

### Initalize type of segmentation to run for building footprint

In [2]:
segmentation = init_segmentation("binary")

### Defining augmenters to use

In [3]:
augmenters = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    
    # 추가
    A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=0.5, border_mode=0),
    
    A.PadIfNeeded(min_height=128, min_width=128, always_apply=True, border_mode=0),
    A.RandomCrop(height=128, width=128, always_apply=True),

    A.OneOf(
    [
        A.CLAHE(p=1),
        A.RandomBrightness(p=1),
        A.RandomGamma(p=1),
    ],
    p=0.8,
    ),
    
    A.OneOf(
        [
            A.Blur(blur_limit=3, p=1),
            A.MotionBlur(blur_limit=3, p=1),
        ],
        p=0.8,
    ),
    
    A.OneOf(
        [
            A.RandomContrast(p=1),
            A.HueSaturationValue(p=1),
        ],
        p=0.8,
    ),
    A.Resize(256, 256),
])

c:\Users\pc\anaconda3\envs\SW\Lib\site-packages\albumentations\augmentations\transforms.py:1258: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
c:\Users\pc\anaconda3\envs\SW\Lib\site-packages\albumentations\augmentations\transforms.py:1284: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


### Load essential segmentation objects for training

In [4]:
model = segmentation.load_model(name="ReFineNet")
criterion = segmentation.load_criterion(name="Dice",dice_weight=0.7)


c:\Users\pc\anaconda3\envs\SW\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\pc\anaconda3\envs\SW\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
loader = segmentation.load_loader(
    root_folder=r"Data_original",
    image_normalization="divide_by_255",
    label_normalization="binary_label",
    augmenters=augmenters,
    batch_size=8,
)


In [6]:
metrics = segmentation.load_metrics(
    data_metrics=["precision", "f1", "recall", "iou", "dice"]
)

# optimizer = segmentation.load_optimizer(model, name="Adam")
optimizer = segmentation.load_optimizer(model, name="SGD",lr=0.05)

callbacks = CallbackList()
# Ouptut from all the callbacks caller will be stored at the path specified in log_dir
for caller in  ["TrainChkCallback", "TimeCallback", "TensorBoardCallback", "TrainStateCallback"]:
    callbacks.append(load_callback(r"out_data", caller))

### Initalize the Trainer object


In [7]:
trainer = Trainer(
    model=model,
    criterion=criterion,
    loader=loader,
    metrics=metrics,
    callbacks=callbacks,
    optimizer=optimizer,
    scheduler=None,
)

### Start training


In [8]:
trainer.train(start_epoch=0, end_epoch=70)

Epoch 0, lr 0.05: 100%|██████████| 7072/7072 [08:24<00:00, 14.02it/s, loss=0.40497]


>>Py-OneLiner : Loss : train: 0.4049731016159058, validation: 0.5039321035146713
>>Py-OneLiner : Train Metric : precision : 0.40211, f1 : 0.36254, recall : 0.41975, iou : 0.24057, dice : 0.36254
>>Py-OneLiner : Valid Metric : precision : 0.72567, f1 : 0.40238, recall : 0.28090, iou : 0.25529, dice : 0.40238

Epoch 1, lr 0.05: 100%|██████████| 7072/7072 [07:30<00:00, 15.69it/s, loss=0.35681]


>>Py-OneLiner : Loss : train: 0.3568114930391312, validation: 0.36488907039165497
>>Py-OneLiner : Train Metric : precision : 0.59868, f1 : 0.56077, recall : 0.59032, iou : 0.40532, dice : 0.56077
>>Py-OneLiner : Valid Metric : precision : 0.58897, f1 : 0.58636, recall : 0.58916, iou : 0.41719, dice : 0.58636

Epoch 2, lr 0.05: 100%|██████████| 7072/7072 [07:15<00:00, 16.23it/s, loss=0.32542]


...

Epoch 63, lr 0.05: 100%|██████████| 7072/7072 [22:22<00:00,  5.27it/s, loss=0.24388]


>>Py-OneLiner : Loss : train: 0.24387747704982757, validation: 0.2640727497637272
>>Py-OneLiner : Train Metric : precision : 0.75500, f1 : 0.72948, recall : 0.73221, iou : 0.59381, dice : 0.72948
>>Py-OneLiner : Valid Metric : precision : 0.66841, f1 : 0.70738, recall : 0.75308, iou : 0.54856, dice : 0.70738

Epoch 64, lr 0.05:   7%|▋         | 472/7072 [01:41<23:36,  4.66it/s, loss=0.22756]



>>Py-OneLiner : KeyBoard Interrupt : State Saved at epoch 64

KeyboardInterrupt: 

### Resuming Training


In [8]:
path_to_saved_state = r"out_data/20230727-005217/state/default.pt"

# if new_end_epoch is None, the trainer will resume and run till end_epoch, the value of which is saved in state file 
trainer.resume(state=path_to_saved_state, new_end_epoch=70)

Epoch 64, lr 0.05: 100%|██████████| 7072/7072 [07:41<00:00, 15.32it/s, loss=0.21811]


>>Py-OneLiner : Loss : train: 0.2181123558431864, validation: 0.22453648038208485
>>Py-OneLiner : Train Metric : precision : 0.76687, f1 : 0.73257, recall : 0.72325, iou : 0.59307, dice : 0.73257
>>Py-OneLiner : Valid Metric : precision : 0.72550, f1 : 0.74938, recall : 0.77672, iou : 0.60149, dice : 0.74938

Epoch 65, lr 0.05: 100%|██████████| 7072/7072 [07:30<00:00, 15.70it/s, loss=0.22118]


>>Py-OneLiner : Loss : train: 0.22118433173745872, validation: 0.24284967966377735
>>Py-OneLiner : Train Metric : precision : 0.75739, f1 : 0.73149, recall : 0.73467, iou : 0.59400, dice : 0.73149
>>Py-OneLiner : Valid Metric : precision : 0.70907, f1 : 0.72671, recall : 0.74683, iou : 0.57373, dice : 0.72671

Epoch 66, lr 0.05: 100%|██████████| 7072/7072 [08:06<00:00, 14.55it/s, loss=0.27258]


...

Epoch 68, lr 0.05: 100%|██████████| 7072/7072 [38:33<00:00,  3.06it/s, loss=0.22727]


>>Py-OneLiner : Loss : train: 0.22727415218949318, validation: 0.24050509929656982
>>Py-OneLiner : Train Metric : precision : 0.74926, f1 : 0.71770, recall : 0.71668, iou : 0.57739, dice : 0.71770
>>Py-OneLiner : Valid Metric : precision : 0.70835, f1 : 0.73008, recall : 0.75611, iou : 0.57697, dice : 0.73008

Epoch 69, lr 0.05:  33%|███▎      | 2336/7072 [25:52<52:27,  1.50it/s, loss=0.21028]  



>>Py-OneLiner : KeyBoard Interrupt : State Saved at epoch 69

KeyboardInterrupt: 